# Spark

Last updated 20161125
    
## 목적

* Spark를 사용하여 빅데이터를 ETL 할 수 있다.
* Spark를 사용하여 분석을 할 수 있다.

## 문제

* 문제 S-1: Spark를 standalone cluster로 구성하기
* 문제 S-2: Hello Spark - 환경설정을 읽어 클라이언트 sc를 생성하기.
* 문제 S-3: RDD를 사용하여 MLlib의 입력 데이터 word vector 생성하기.
* 문제 S-4: RDD를 사용하여 MLLib의 입력 데이터 feature vector 생성하기.
* 문제 S-5: 파일에서 Spark SQL로 데이터 읽기
* 문제 S-6: spark sql uber csv
* 문제 S-7: Kolmogorov-Smirnov 검증
* 문제 S-8: 무작위 데이터 생성
* 문제 S-9: 정량데이터 분석
* 문제 S-10: 텍스트 분석
* 문제 S-11: twitter 데이터 분석
* 문제 S-12: 그래프 분석
* 문제 S-13: spark-submit
* 문제 S-14: 시각화 Bokeh
http://www.blog.pythonlibrary.org/2016/07/27/python-visualization-with-bokeh/

* ref
    * introduction to big data with apache spark (Berkley Anthony Joseph)
    * [spark-sklearn](https://github.com/databricks/spark-sklearn)
        ```
        pip install spark-sklearn
        ```
        
        * spark-shell
        ```
        $SPARK_HOME/bin/spark-shell --packages databricks:spark-sklearn:0.2.0
        ```
        
    * 듀크대학 STA663 Statistical Computing and Computation, Spring 2016
        * https://github.com/cliburn/sta-663-2016

    * kaggle
        * https://www.kaggle.com/kaggle/us-baby-names

## S.1 빅데이터와 Spark

* 빅데이터는 여러 출처에서 발생하며, 그 형식이 비구조적이고 다양하다.
* 데이터 웨어하우스, 데이터 마이닝과 관련이 있다.
* ETL (Extract, Transfrom and Load)
    * RapidMiner, SAS 등

단계 | 설명
-----|-----
Extract | 다양한 소스에서 데이터 추출 (Hadoop files, files, json, DB...)
Transform | 데이터 변환.
Load | 데이터베이스에 저장하여 사용

## S.2 Spark 소개

### 버전
* 2009년 UC Berkeley, Matei Zaharia 박사과정 하면서 개뱔
* 2010년 BSD 라이센스 오픈소스로 전환.
* 2013년 Apache 2.0 license로 전환
* 현재 개발자가 Databricks를 설립해서 관리

### 왜 Spark를 배워야 하는가?
* REPL (Read Eval Print Looop)이 가능해서 배우기 쉽다. 쉘 환경이 있어 편리하다. Standalone으로 시작할 수 있다.
* 빅데이터를 빠르게 map reduce 할 수 있다.
* Machine learning 라이브러리를 가지고 있다.

### 구조

* 분산 클러스터 컴퓨팅 프레임워크로서, API를 사용해서 데이터추출, 변환, 기계학습, 그래프분석을 할 수 있다.
* Spark Core가 분산작업에 필요한 바탕이 되고, 그 위 sql, streaming, mllib, graphx를 제공한다.

구분 | 구성 | 설명
-------|-------|-------
Spark engine | Spark Core | 작업배분, 입출력 등 분산작업에 필요한 기능
Spark Applicaiton Frameworks | Spark SQL | DataFrames
| Spark Streaming | 실시간 처리
| MLlib | 머신러닝 (참조 scikit-learn)
|GraphX | 그래프 분석


* 빅데이터를 처리하기 위해 만들어져 있고, Hadoop과 달리 메모리에서 처리하기 때문에 빠름 (pipeline). Spark는 RDD를 통해 Hadoop을 사용할 수 있다.
* Scala로 개발되어 jvm에서 실행. 그러나 Scala, Java, Python, R 여러 언어를 섞어서 할 수 있는 환경을 제공 (polyglot)

구분 | Spark | Hadoop
-------|-------|-------
사용 목적 | 데이터 분석 | 데이터 분산 처리
파일 시스템 | 자체 파일 시스템이 없슴. hdfs, db, csv등을 사용 | hdfs
속도 | 파이프라인을 사용하므로 빠름 | 보다 느림


### 설치

* 설치하려는 하둡의 버전을 선택하여, prebuilt distribution을 설치한다.
* [Spark 다운로드](https://spark.apache.org/downloads.html)
    * spark 1.6 hadoop2.6 
        ```
        tar xvfz spark-1.x.x-bin-hadoop2.x.tgz
        ```


## 문제 S-1: Spark를 standalone cluster로 구성하기

* 클러스터를 구성하지 않으면 클러스터 없이 운영 - curl로 7077, 8080확인해도 없음 -> NO cluster!

* 클러스터의 종류:
    * Spark-Standalone – Spark workers are registered with spark master
    * Yarn – Spark workers are registered with YARN Cluster manager.
    * Mesos – Spark workers are registered with Mesos.

* 클러스터 환경 구성
    * Client: spark shell, pyspark shell
    * Cluster
        * Cluster 1:
            * Spark master / Spark worker
            * hdfs namenode / datanode
        * Cluster n:
            * Spark worker
            * hdfs datanode

*  Spark-Standalone
    * SPARK_HOME은 /home/jsl/Downloads/spark-1.6.0-bin-hadoop2.6/
    * 단계1 JAVA_HOME을 설정
        * JAVA_HOME을 설정한다.
            * automatically set symlink to java binary /usr/bin/java
            * JAVA_HOME을 설정하려면 /etc/environment에 하는 것이 좋다.
            ```
            $ echo $JAVA_HOME
            $ update-alternatives --config java
            ```

    * 단계2: master 실행
        * $SPARK_HOME/conf/spark-env.sh에 master ip설정
            ```
            SPARK_MASTER_IP=
            ```

        * 실행
            ```
            $ sh $SPARK_HOME/sbin/start-master.sh
            ```
            * spark://IPADRESS_OF_YOUR_MASTER_SYSTEM:7077
            * 기본 port는 7077 (web UI는 localhost:8080)

    * 단계3: slave 실행 (worker라고 함)
        ```
        $ sh $SPARK_HOME/sbin//start-slave.sh spark://IPADRESS_OF_YOUR_MASTER_SYSTEM:7077
        ```

* 쉘 명령어

sbin디렉토리의 shell | 설명
----------|----------
start-master.sh, stop-master.sh | 마스터를 시작 (종료)
start-slaves.sh, stop-slaves.sh | 각 노드의 슬레이브를 시작 (종료)
start-all.sh, stop-all.sh | 마스터, 슬레이브를 모두 시작 (종료)

## 문제 S-2: Hello Spark: 환경설정을 읽어 클라이언트 sc를 생성하기.

* Spark는 batch, streaming, iterative, interactive 4가지 방식으로 실행할 수 있다.
* 환경설정을 읽어 sc를 생성하고, 이를 클라이언트와 같이 Spark를 사용한다.

### spark-submit
* spark 프로그램을 일괄 실행

### insteractive shell

* scala
    ```
    ./bin/spark-shell
    scala>
    ```

* python
    ```
    spark-1.6.0-bin-hadoop2.6/bin$ pyspark
    Welcome to
          ____              __
         / __/__  ___ _____/ /__
        _\ \/ _ \/ _ `/ __/  '_/
       /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
          /_/

    Using Python version 2.7.12 (default, Jul  1 2016 15:12:24)
    SparkContext available as sc, HiveContext available as sqlContext.
    >>> sc.version
    u'1.6.0'
    >>> text=sc.textFile("derby.log");
    >>>
    ```


* SparkContext
    * Spark서버(클러스터)에 대한 클라이언트와 같은 역할로, 반드시 있어야 한다.
    * 클러스터를 어떻게 사용할 것인지 정하는 것 -> cluster manager에서 system resource를 할당받음 (cpu, memory, machine)
    * Python의 SparkContext는 jar를 분산환경에서 사용하게 되므로 주의 (Scala, Java와 다름)
        * pyFiles에 사용할 (의존적인) 라이브러리를 넣는다 (또는 사용할 라이브러리가 없으면 빈 파일로 둔다).

    * Cannot run multiple SparkContexts at once;
        * sc가 이미 있는 경우
        ```
        from pyspark import SparkContext
        sc = SparkContext("master","my python app", sparkHome="sparkhome",pyFiles="placeholderdeps.zip")
        ```

* SparkConf
    * spark-defaults.conf와 같은 파일의 값을 읽어서 설정
    ```
    scala> sc.getConf.getOption("spark.local.dir")
    res0: Option[String] = None

    scala> sc.getConf.getOption("spark.app.name")
    res1: Option[String] = Some(Spark shell)

    scala> sc.getConf.get("spark.master")
    res2: String = local[*]
    ```

### PySpark on IPython Notebook

* ipython notebook에서 pyspark를 사용
* kernel을 만들지 않고, findspark를 사용한다.
* SPARK_HOME을 설정해서 사용한다
    * 현재 ~/Downloads/spark-1.6.0-bin-hadoop2.6
* kernel을 사용하면 아래와 같이 한다.
    ```
    from pyspark import SparkContext, SparkConf
    conf = SparkConf().setAppName("jsl").setMaster("local")
    sc = SparkContext(conf=conf)
    ```

In [1]:
import os
import findspark

home=os.getenv("HOME")
spark_home=os.path.join(home,"Downloads/spark-1.6.0-bin-hadoop2.6")
findspark.init(spark_home)

* SparkConf()에서 설정한 값을 읽어 SparkContext sc를 생성한다.
    * spark.master와 spark.app.name 필수적으로 설정해야 한다.
* standalone 실행에서의 spark master
    * local로 설정되고, 기본은 필요한만큼 쓰레드를 생성한다 (local[*])

In [2]:
import pyspark
conf=pyspark.SparkConf()
conf = pyspark.SparkConf().setAppName("myAppName")
sc = pyspark.SparkContext(conf=conf)

In [3]:
print sc

In [4]:
sc.version

u'1.6.0'

* 설정을 읽어 온다 (conf디렉토리 아래)  

In [3]:
sc.master

u'local[*]'

In [7]:
sc._conf.get("spark.jars.packages")

u'graphframes:graphframes:0.1.0-spark1.6,org.mongodb.spark:mongo-spark-connector_2.10:1.1.0'

In [4]:
sc._conf.getAll()

[(u'spark.app.name', u'myAppName'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.jars',
  u'file:/home/jsl/.ivy2/jars/graphframes_graphframes-0.1.0-spark1.6.jar,file:/home/jsl/.ivy2/jars/org.mongodb.spark_mongo-spark-connector_2.10-1.1.0.jar,file:/home/jsl/.ivy2/jars/org.mongodb_mongo-java-driver-3.2.2.jar'),
 (u'spark.jars.packages',
  u'graphframes:graphframes:0.1.0-spark1.6,org.mongodb.spark:mongo-spark-connector_2.10:1.1.0'),
 (u'spark.files',
  u'file:/home/jsl/.ivy2/jars/graphframes_graphframes-0.1.0-spark1.6.jar,file:/home/jsl/.ivy2/jars/org.mongodb.spark_mongo-spark-connector_2.10-1.1.0.jar,file:/home/jsl/.ivy2/jars/org.mongodb_mongo-java-driver-3.2.2.jar'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.master', u'local[*]'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.submit.pyFiles',
  u'/home/jsl/.ivy2/jars/graphframes_graphframes-0.1.0-spark1.6.jar,/home/jsl/.ivy2/jars/org.mongodb.spark_mongo-spark-connector_2.10-1.1.0.jar,/home/jsl/.ivy2/jars/org.mong

## S.3 Hello RDD

* RDD (Resilient Distributed Dataset)는 분산데이터 형식이다.
    * Resilient - fault tolerent
    * Distributed - multiple nodes in a clusters
    * Dataset - 데이터타잎으로 구성된다.
* RDD는 내, 외부 자료에서 생성하며, 생성된 자료는 read-only이다.
    * HDFS 파일을 처리할 수 있다.

* 3단계 처리
    * 1단계: 읽기 - 2가지 방식: 
        * 외부에서 읽기 
        ```
        sc.textFile()
        ```
        
        * 내부에서 읽기 parallelizing a collection
        ```
        sc.parallelize()
        ```
        
    * 2단계: 변환 transformations - lazy도 가능: RDD => RDD or seq(RDD)

함수 | 설명 | 예제
-------|-------|-------
map(fn) | 요소별로 fn을 적용해서 결과 rdd 돌려줌 | map
filter(fn) | 요소별로 선별하여 fn을 적용해서 결과 rdd 돌려줌 | filter(lambda line: "Spark" in line)
flatMap(fn) | 요소별로 fn을 적용하고, flat해서 결과 rdd 돌려줌 | .flatMap(lambda x: x.split(' '))
groupByKey() | |


    * 3단계: actions: RDD => a value (e.g., python list)

함수 | 설명 | 예제
-------|-------|-------
reduce(fn) | 요소별로 fn을 사용해서 줄여서 결과 list를 돌려줌 |
collect() | 모든 요소를 결과 list로 돌려줌 |
count() | 요소의 갯수를 결과 list로 돌려줌 |
countByKey() | |
foreach(fn) | |



### Spark를 사용하기 전, python 함수 사용하기

* map, reduce, filter
    * 함수의 인자는 2개가 필요하다 (함수, 데이터).

함수 | 설명 | 예
-------|-------|-------
map() | 각 데이터 요소에 함수를 적용해서 list를 반환 | map(fn,data)
filter() | 각 데이터 요소에 함수의 결과 True를 선택해서 반환 | filter(fn, data)
reduce() | 각 데이터 요소에 함수를 적용해서 list를 반환 | reduce(fn, data)

* Python 함수로 처리한다.
    * 입출력은 데이터 하나씩이 아니라, list로 한다.

In [28]:
celsius = [39.2, 36.5, 37.3, 37.8]
def c2f(c):
    f=list()
    for i in c:
        _f=(float(9)/5)*i + 32
        f.append(_f)
    return f

print c2f(celsius)

[102.56, 97.7, 99.14, 100.03999999999999]


* Python에서 제공하는 map() 함수를 사용한다. map() 함수의 인자:
    * (1) 함수명 (함수의 return은 반드시 있어야 한다.)
    * (2) 입력인자

In [8]:
celsius = [39.2, 36.5, 37.3, 37.8]

def c2f(c):
    return (float(9)/5)*c + 32

f=map(c2f, celsius)
print f

[102.56, 97.7, 99.14, 100.03999999999999]


* lambda함수를 사용한다.
    * lambda는 무명 함수이다. 처리 결과가 반환된다.

In [9]:
map(lambda c:(float(9)/5)*c + 32, celsius)

[102.56, 97.7, 99.14, 100.03999999999999]

* 문자열에 map()을 사용한다.

In [14]:
sentence = 'Hello World'
words = sentence.split()
print words

['Hello', 'World']


* 문자열을 사용하면, 각 단어를 split()한다.
* list를 사용하면, 각 요소를 split()한다.

In [17]:
sentence = "Hello World"
map(lambda x:x.split(),sentence)

[['H'], ['e'], ['l'], ['l'], ['o'], [], ['W'], ['o'], ['r'], ['l'], ['d']]

In [18]:
sentence = ["Hello World"]
map(lambda x:x.split(),sentence)

[['Hello', 'World']]

* filter()는 데이터를 선별한다.

In [37]:
fib = [0,1,1,2,3,5,8,13,21,34,55]
result = filter(lambda x: x % 2, fib)
print result

[1, 1, 3, 5, 13, 21, 55]


* reduce()는 2개의 인자를 받는다.
* [ func(func(s1, s2),s3), ... , sn ]와 같이 수행한다.

In [36]:
reduce(lambda x, y: x+y, range(1,101))

5050

### Spark RDD 사용하기

* Apache spark wiki에서 첫 문단을 복사해 왔다.
* 3째줄은 한글, 4째 줄은 같은 단어를 반복해 추가했다.


In [528]:
%%writefile data/ds_spark_wiki.txt
Wikipedia
Apache Spark is an open source cluster computing framework.
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
Apache Spark Apache Spark Apache Spark Apache Spark
Originally developed at the University of California, Berkeley's AMPLab,
the Spark codebase was later donated to the Apache Software Foundation,
which has maintained it since.
Spark provides an interface for programming entire clusters with
implicit data parallelism and fault-tolerance.

Overwriting data/ds_spark_wiki.txt


* 파일에서 읽기
    ```
    textFile()
    ```

In [2]:
textFile = sc.textFile("data/ds_spark_wiki.txt")

In [3]:
def mySplit(x):
    return x.split(" ")

words=textFile.map(mySplit)

In [448]:
type(words)

pyspark.rdd.PipelinedRDD

In [449]:
words.count()

8

In [450]:
for i in words.collect():
    print i

[u'Wikipedia']
[u'Apache', u'Spark', u'is', u'an', u'open', u'source', u'cluster', u'computing', u'framework.']
[u'\uc544\ud30c\uce58', u'\uc2a4\ud30c\ud06c\ub294', u'\uc624\ud508', u'\uc18c\uc2a4', u'\ud074\ub7ec\uc2a4\ud130', u'\ucef4\ud4e8\ud305', u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.']
[u'Originally', u'developed', u'at', u'the', u'University', u'of', u'California,', u"Berkeley's", u'AMPLab,']
[u'the', u'Spark', u'codebase', u'was', u'later', u'donated', u'to', u'the', u'Apache', u'Software', u'Foundation,']
[u'which', u'has', u'maintained', u'it', u'since.']
[u'Spark', u'provides', u'an', u'interface', u'for', u'programming', u'entire', u'clusters', u'with']
[u'implicit', u'data', u'parallelism', u'and', u'fault-tolerance.']


In [5]:
textFile.first()

u'Wikipedia'

* map()함수로 단어 분리하기

In [15]:
words=textFile.map(lambda x:x.split(' '))

In [16]:
words.collect()

[[u'Wikipedia'],
 [u'Apache',
  u'Spark',
  u'is',
  u'an',
  u'open',
  u'source',
  u'cluster',
  u'computing',
  u'framework.'],
 [u'\uc544\ud30c\uce58',
  u'\uc2a4\ud30c\ud06c\ub294',
  u'\uc624\ud508',
  u'\uc18c\uc2a4',
  u'\ud074\ub7ec\uc2a4\ud130',
  u'\ucef4\ud4e8\ud305',
  u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.'],
 [u'Originally',
  u'developed',
  u'at',
  u'the',
  u'University',
  u'of',
  u'California,',
  u"Berkeley's",
  u'AMPLab,'],
 [u'the',
  u'Spark',
  u'codebase',
  u'was',
  u'later',
  u'donated',
  u'to',
  u'the',
  u'Apache',
  u'Software',
  u'Foundation,'],
 [u'which', u'has', u'maintained', u'it', u'since.'],
 [u'Spark',
  u'provides',
  u'an',
  u'interface',
  u'for',
  u'programming',
  u'entire',
  u'clusters',
  u'with'],
 [u'implicit', u'data', u'parallelism', u'and', u'fault-tolerance.']]

* 각 문장의 철자 갯수를 센다.
    * 첫 문장 'Wikipedia'는 9

In [24]:
textFile.map(lambda s:len(s)).collect()

[9, 59, 32, 72, 71, 30, 64, 46]

* filter()

In [6]:
_sparkLine=textFile.filter(lambda line: "Spark" in line)

In [25]:
print _sparkLine.count()

3


* 한글은 앞에 u를 붙여준다.

In [26]:
_line = textFile.filter(lambda line: u"스파크" in line)

In [27]:
print _line.first()

아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.


* parallelize() 사용하기
    * list에서 읽어, rdd로 변환하기

In [28]:
_aList=[1,2,3]
rdd = sc.parallelize(_aList)

In [44]:
rdd.take(3)

[1, 2, 3]

* map(), collect() 사용해서 square

In [29]:
nums = sc.parallelize([1, 2, 3, 4])
squared = nums.map(lambda x: x * x).collect()
print squared

[1, 4, 9, 16]


* 문장 처리하기
* 단어를 교체하기

In [23]:
a=["this is","a line"]
_rdd=sc.parallelize(a)

words=_rdd.map(lambda x:x.split())
print words.collect()

[['this', 'is'], ['a', 'line']]


In [18]:
_upper=_rdd.map(lambda x:x.replace("a","AA"))
_upper.take(10)

['this is', 'AA line']

* 첫 글자를 대문자로 만들어서 출력해 보기

In [29]:
's'.upper()

'S'

In [30]:
pluralRDD =words.map(lambda x: x[0].upper())
print pluralRDD.collect()

['THIS', 'A']


In [4]:
pluralRDD =words.map(lambda x: [i.upper() for i in x])
print pluralRDD.collect()

[[u'WIKIPEDIA'], [u'APACHE', u'SPARK', u'IS', u'AN', u'OPEN', u'SOURCE', u'CLUSTER', u'COMPUTING', u'FRAMEWORK.'], [u'\uc544\ud30c\uce58', u'\uc2a4\ud30c\ud06c\ub294', u'\uc624\ud508', u'\uc18c\uc2a4', u'\ud074\ub7ec\uc2a4\ud130', u'\ucef4\ud4e8\ud305', u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.'], [u'APACHE', u'SPARK', u'APACHE', u'SPARK', u'APACHE', u'SPARK', u'APACHE', u'SPARK'], [u'ORIGINALLY', u'DEVELOPED', u'AT', u'THE', u'UNIVERSITY', u'OF', u'CALIFORNIA,', u"BERKELEY'S", u'AMPLAB,'], [u'THE', u'SPARK', u'CODEBASE', u'WAS', u'LATER', u'DONATED', u'TO', u'THE', u'APACHE', u'SOFTWARE', u'FOUNDATION,'], [u'WHICH', u'HAS', u'MAINTAINED', u'IT', u'SINCE.'], [u'SPARK', u'PROVIDES', u'AN', u'INTERFACE', u'FOR', u'PROGRAMMING', u'ENTIRE', u'CLUSTERS', u'WITH'], [u'IMPLICIT', u'DATA', u'PARALLELISM', u'AND', u'FAULT-TOLERANCE.']]


* transformation(map()), action(collect()) 함수를 한꺼번에

In [33]:
pluralRDD =words.map(lambda x: [i.upper() for i in x]).collect()
print pluralRDD

[['THIS', 'IS'], ['A', 'LINE']]


In [34]:
wordsLength = words\
    .map(len)\
    .collect()
print wordsLength

[2, 2]


* 파일에 쓰기

In [6]:
pluralRDD.saveAsTextFile("data/ds_spark_wiki1.txt")

* create RDD from CSV


In [6]:
%%writefile ./data/ds_spark_2cols.csv
35, 2
40, 27
12, 38
15, 31
21, 1
14, 19
46, 1
10, 34
28, 3
48, 1
16, 2
30, 3
32, 2
48, 1
31, 2
22, 1
12, 3
39, 29
19, 37
25, 2


Writing ./data/ds_spark_2cols.csv


In [6]:
inp_file = sc.textFile("./data/ds_spark_2cols.csv")
numbers_rdd = inp_file.map(lambda line: line.split(','))

In [7]:
numbers_rdd.take(10)

[[u'35', u' 2'],
 [u'40', u' 27'],
 [u'12', u' 38'],
 [u'15', u' 31'],
 [u'21', u' 1'],
 [u'14', u' 19'],
 [u'46', u' 1'],
 [u'10', u' 34'],
 [u'28', u' 3'],
 [u'48', u' 1']]

## S.4 데이터 타잎

구분 | 설명
----------|----------
Vector | dense와 sparse가 있다.
Labled Point | 클래스값과 결과를 묶음. supervised learning에 사용.
Matrix | 행열로 구성

* vectors
    * local vector - single machine에 있다.
    * pyspark.mllib.linalg.Vectors

dense vector | sparse vector
----------|----------
빈 값이 별로 없는 경우. an array of its values | 빈 값이 많은 경우 사용. 인덱스, 값 배열 별도
(160,69,24) | (3,[0,1,2],[160.0,69.0,24.0])
입력 NumPy’s array, Python list | MLlib’s SparseVector, SciPy’s csc_matrix with a single column

### dense vector

In [41]:
from pyspark.mllib.linalg import Vectors
dv1=Vectors.dense([0.0, 1.1, 0.1])

In [42]:
print dv1

[0.0,1.1,0.1]


* numpy를 사용해서 dense vector를 생성할 수 있다.

In [45]:
import numpy as np

dv2 = np.array([1.0, 2.1, 3.2])

In [43]:
print dv1

[0.0,1.1,0.1]


* Python list를 사용하여 dense vector를 생성할 수 있다.

In [46]:
dv2 = [1.0, 2.1, 3.2]

### sparse vector.

* SparseVector vs Vectors.sparse 차이?

* toArray()는 1줄씩


In [48]:
sv1 = Vectors.sparse(3, [1, 2], [1.0, 3.0])
print sv1.toArray()

[ 0.  1.  3.]


* scipy.sparse

In [50]:
sv2 = sps.csc_matrix((np.array([1.0,3.0]), np.array([0,2]), np.array([0,2])), shape = (3,1))
sv2.todense()
print sv2

  (0, 0)	1.0
  (2, 0)	3.0


### labeled point

* local vector, either dense or sparse
* label과 response로 구성된다.


In [52]:
from pyspark.mllib.regression import LabeledPoint
LabeledPoint(1, [1.0, 2.0, 3.0])

LabeledPoint(1.0, [1.0,2.0,3.0])

In [8]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
LabeledPoint(1.0, Vectors.dense([1.0, 2.0, 3.0]))

LabeledPoint(1.0, [1.0,2.0,3.0])

* dataframe 생성하기

In [184]:
from pyspark.mllib.linalg import Vectors

trainDf = sqlCtx.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, 1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, 0.5]))], ["label", "features"])

* schema를 사용해서 dataframe 생성하기

In [ ]:
from pyspark.mllib.linalg import SparseVector, VectorUDT
from pyspark.sql.types import StructType, StructField, DoubleType
_rdd = sc.parallelize([
    (0.0, SparseVector(4, {1: 1.0, 3: 5.5})),
    (1.0, SparseVector(4, {0: -1.0, 2: 0.5}))])

In [ ]:
schema = StructType([
    StructField("label", DoubleType(), True),
    StructField("features", VectorUDT(), True)
])

In [188]:
_df=_rdd.toDF(schema)
_df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



### maxtrix

* local matrix - pyspark.mllib.linalg.Matrix, Matrices
* distributed matrix
    * pyspark.mllib.linalg.distributed.RowMatrix
    * pyspark.mllib.linalg.distributed.IndexedRow, IndexedRowMatrix
    * pyspark.mllib.linalg.distributed.BlockMatrix

In [53]:
from pyspark.mllib.linalg import Matrix, Matrices

# Create a dense matrix ((1.0, 2.0), (3.0, 4.0), (5.0, 6.0))
dm = Matrices.dense(3, 2, [1, 2, 3, 4, 5, 6])

# Create a sparse matrix ((9.0, 0.0), (0.0, 8.0), (0.0, 6.0))
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 2, 1], [9, 6, 8])

### libsvm format

data = MLUtils.loadLibSVMFile(sc, 'data/mllib/sample_libsvm_data.txt')
* format
    ```
    [label] [index1]:[value1] [index2]:[value2] ...
    [label] [index1]:[value1] [index2]:[value2] ...
    ```
    * label - class
    * index - integers
    * value - real numbers

* see - /home/jsl/Downloads/spark-1.6.0-bin-hadoop2.6/data/mllib/sample_libsvm_data.txt
```
0 128:51 129:159 130:253 131:159 132:50 155:48 156:238 157:252 158:252 159:252 160:237 182:54 183:227 184:253 185:252 186:239 187:233 188:252 189:57 190:6 208:10 209:60 210:224 211:252 212:253 213:252 214:202 215:84 216:252 217:253 218:122 236:163 237:252 238:252 239:252 240:253 241:252 242:252 243:96 244:189 245:253 246:167 263:51 264:238 265:253 266:253 267:190 268:114 269:253 270:228 271:47 272:79 273:255 274:168 290:48 291:238 292:252 293:252 294:179 295:12 296:75 297:121 298:21 301:253 302:243 303:50 317:38 318:165 319:253 320:233 321:208 322:84 329:253 330:252 331:165 344:7 345:178 346:252 347:240 348:71 349:19 350:28 357:253 358:252 359:195 372:57 373:252 374:252 375:63 385:253 386:252 387:195 400:198 401:253 402:190 413:255 414:253 415:196 427:76 428:246 429:252 430:112 441:253 442:252 443:148 455:85 456:252 457:230 458:25 467:7 468:135 469:253 470:186 471:12 483:85 484:252 485:223 494:7 495:131 496:252 497:225 498:71 511:85 512:252 513:145 521:48 522:165 523:252 524:173 539:86 540:253 541:225 548:114 549:238 550:253 551:162 567:85 568:252 569:249 570:146 571:48 572:29 573:85 574:178 575:225 576:253 577:223 578:167 579:56 595:85 596:252 597:252 598:252 599:229 600:215 601:252 602:252 603:252 604:196 605:130 623:28 624:199 625:252 626:252 627:253 628:252 629:252 630:233 631:145 652:25 653:128 654:252 655:253 656:252 657:141 658:37
```

In [10]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

In [11]:
svmfn="/home/jsl/Downloads/spark-1.6.0-bin-hadoop2.6/data/mllib/sample_libsvm_data.txt"
svmDf = sqlCtx.read.format("libsvm").load(svmfn)

In [12]:
type(svmDf)

pyspark.sql.dataframe.DataFrame

In [13]:
svmDf.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = false)



* csr format (https://www.ncsu.edu/hpc/Courses/6sparse.html)
    ```
    0 0 0 0
    5 8 0 0
    0 0 3 0
    0 6 0 0
    ```
    * non-zero 5 8 3 6
    * column-index 0 1 2 1 (5(1,0) 8(1,1) 3(2,2) 6(3,1)에서 행값만 추출)



## 문제 S-3: RDD를 사용하여 MLlib의 입력 데이터 word vector생성하기.

* RDD API를 사용해서 단어를 셀 수 있다 (map, reduce 등).
* mllib 패키지를 사용하여 데이터를 변환할 수 있다.
    * TF-IDF, Word2Vec 등을 사용할 수 있다.
    * mllib에 없는 변환기능은 ml을 사용한다 (ml은 dataframe을 변환하는 패키지.)
        * Tokenizer, StopWordsRemove, n-gram등


In [439]:
!ls data/ds_spark_wiki.txt

data/ds_spark_wiki.txt


### 파일 전체 word count

In [19]:
lines = sc.textFile("data/ds_spark_wiki.txt")
wc = lines\
    .flatMap(lambda x: x.split(' '))

In [20]:
wc.collect()

[u'Wikipedia',
 u'Apache',
 u'Spark',
 u'is',
 u'an',
 u'open',
 u'source',
 u'cluster',
 u'computing',
 u'framework.',
 u'\uc544\ud30c\uce58',
 u'\uc2a4\ud30c\ud06c\ub294',
 u'\uc624\ud508',
 u'\uc18c\uc2a4',
 u'\ud074\ub7ec\uc2a4\ud130',
 u'\ucef4\ud4e8\ud305',
 u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.',
 u'Apache',
 u'Spark',
 u'Apache',
 u'Spark',
 u'Apache',
 u'Spark',
 u'Apache',
 u'Spark',
 u'Originally',
 u'developed',
 u'at',
 u'the',
 u'University',
 u'of',
 u'California,',
 u"Berkeley's",
 u'AMPLab,',
 u'the',
 u'Spark',
 u'codebase',
 u'was',
 u'later',
 u'donated',
 u'to',
 u'the',
 u'Apache',
 u'Software',
 u'Foundation,',
 u'which',
 u'has',
 u'maintained',
 u'it',
 u'since.',
 u'Spark',
 u'provides',
 u'an',
 u'interface',
 u'for',
 u'programming',
 u'entire',
 u'clusters',
 u'with',
 u'implicit',
 u'data',
 u'parallelism',
 u'and',
 u'fault-tolerance.']

* 단어를 세어서 tuple로 만듦

In [453]:
from operator import add
wc = sc.textFile("data/ds_spark_wiki.txt")\
    .flatMap(lambda x: x.split(' '))\
    .map(lambda x: (x.lower().rstrip().lstrip().rstrip(',').rstrip('.'), 1))\
    .reduceByKey(add)

In [454]:
wc.count()

50

In [455]:
wc.first()

(u'and', 1)

### 라인 별 word count

* dataframe으로 처리

In [608]:
from operator import add
wc = sc.textFile("data/ds_spark_wiki.txt")\
    .map(lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower())\
    .map(lambda x:x.split())\
    .map(lambda x:[(i,1) for i in x])

In [609]:
for e in wc.collect():
    print e

[(u'wikipedia', 1)]
[(u'apache', 1), (u'spark', 1), (u'is', 1), (u'an', 1), (u'open', 1), (u'source', 1), (u'cluster', 1), (u'computing', 1), (u'framework', 1)]
[(u'\uc544\ud30c\uce58', 1), (u'\uc2a4\ud30c\ud06c\ub294', 1), (u'\uc624\ud508', 1), (u'\uc18c\uc2a4', 1), (u'\ud074\ub7ec\uc2a4\ud130', 1), (u'\ucef4\ud4e8\ud305', 1), (u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4', 1)]
[(u'apache', 1), (u'spark', 1), (u'apache', 1), (u'spark', 1), (u'apache', 1), (u'spark', 1), (u'apache', 1), (u'spark', 1)]
[(u'originally', 1), (u'developed', 1), (u'at', 1), (u'the', 1), (u'university', 1), (u'of', 1), (u'california', 1), (u"berkeley's", 1), (u'amplab', 1)]
[(u'the', 1), (u'spark', 1), (u'codebase', 1), (u'was', 1), (u'later', 1), (u'donated', 1), (u'to', 1), (u'the', 1), (u'apache', 1), (u'software', 1), (u'foundation', 1)]
[(u'which', 1), (u'has', 1), (u'maintained', 1), (u'it', 1), (u'since', 1)]
[(u'spark', 1), (u'provides', 1), (u'an', 1), (u'interface', 1), (u'for', 1), (u'programming'

* TF (Term Frequency)
    * HashingTF

In [21]:
documents = sc.textFile("data/ds_spark_wiki.txt").map(lambda line: line.split(" "))

In [26]:
from pyspark.mllib.feature import HashingTF

hashingTF = HashingTF()
tf = hashingTF.transform(documents)

In [28]:
tf.collect()

[SparseVector(1048576, {253068: 1.0}),
 SparseVector(1048576, {36751: 1.0, 50570: 1.0, 68380: 1.0, 415281: 1.0, 511377: 1.0, 728364: 1.0, 862087: 1.0, 938426: 1.0, 999480: 1.0}),
 SparseVector(1048576, {63234: 1.0, 340190: 1.0, 357478: 1.0, 375592: 1.0, 458138: 1.0, 486171: 1.0, 598772: 1.0}),
 SparseVector(1048576, {938426: 4.0, 999480: 4.0}),
 SparseVector(1048576, {36757: 1.0, 225801: 1.0, 323305: 1.0, 453405: 1.0, 498679: 1.0, 518030: 1.0, 688842: 1.0, 762570: 1.0, 959994: 1.0}),
 SparseVector(1048576, {420843: 1.0, 550676: 1.0, 725041: 1.0, 782544: 1.0, 938426: 1.0, 959994: 2.0, 991590: 1.0, 993084: 1.0, 996703: 1.0, 999480: 1.0}),
 SparseVector(1048576, {50573: 1.0, 263739: 1.0, 892834: 1.0, 1014710: 1.0, 1035538: 1.0}),
 SparseVector(1048576, {3932: 1.0, 36751: 1.0, 192182: 1.0, 358969: 1.0, 363244: 1.0, 496856: 1.0, 546913: 1.0, 938426: 1.0, 951974: 1.0}),
 SparseVector(1048576, {69621: 1.0, 157580: 1.0, 219357: 1.0, 297436: 1.0, 715648: 1.0})]